<a href="https://colab.research.google.com/github/vyarca2/Melspectrogram-shape-Python-and-CPP/blob/main/Task_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
!apt-get install -y libsndfile1
!pip install pytest


In [ ]:
!pip install librosa


In [1]:
import numpy as np
from scipy.io import wavfile

duration = 5
sample_rate = 16000
frequency = 440

t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)

audio = np.sin(2 * np.pi * frequency * t)

audio *= 0.3 * np.iinfo(np.int16).max

audio = audio.astype(np.int16)

wavfile.write('/content/audio1.wav', sample_rate, audio)

print("Audio file saved as audio1.wav")


Audio file saved as audio1.wav


In [5]:
from IPython.display import Audio

audio_file = '/content/audio1.wav'

Audio(audio_file)


In [7]:
%%writefile mel.cpp
#include <iostream>
#include <vector>
#include <complex>
#include <string>
#include <fstream>
#include </content/drive/MyDrive/LibrosaCpp-main/librosa/librosa.h>
#include <sndfile.h>

std::vector<float> loadaudio(const std::string& filepath)
{
    SNDFILE* audiofile;
    SF_INFO audiofileinfo;
    audiofile = sf_open(filepath.c_str(), SFM_READ, &audiofileinfo);

    if (!audiofile)
    {
        std::cout << "Failed to open audio file: " << filepath << std::endl;
        return std::vector<float>();
    }

    std::vector<float> audiodata(audiofileinfo.frames);
    sf_read_float(audiofile, audiodata.data(), audiofileinfo.frames);
    sf_close(audiofile);

    return audiodata;
}

int main()
{
    std::string audiofilepath = "/content/audio1.wav";
    std::vector<float> audiodata = loadaudio(audiofilepath);

    if (!audiodata.empty())
    {
        int samplerate = 16000;
        int nfft = 2048;
        int hoplength = 512;
        int nmels = 128;
        int fmin = 0;
        int fmax = samplerate ;

        std::vector<std::vector<float>> melSpectrogram = librosa::Feature::melspectrogram(audiodata, samplerate, nfft, hoplength, "hann", true, "reflect", 1.0f, nmels, fmin, fmax);
        std::ofstream outputfile("mel_spectrogram.txt");

        if (outputfile.is_open())
        {
            outputfile <<"("<< melSpectrogram.size() << ", " << melSpectrogram[0].size()<<")" << std::endl;
            outputfile.close();

            std::cout << "Mel spectrogram shape saved to mel_spectrogram.txt" << std::endl;
        }
        else
        {
            std::cout << "Failed to open the output file" << std::endl;
        }
    }
    else
    {
        std::cout << "Failed to load audio file: " << audiofilepath << std::endl;
    }

    return 0;
}



Writing mel.cpp


In [8]:
!g++ mel.cpp -o mel -lsndfile
!./mel

Mel spectrogram shape saved to mel_spectrogram.txt


In [9]:
import soundfile as sf
import librosa
import numpy as np

def load_audio(file_path):
    audio, sample_rate = sf.read(file_path, dtype='float32')
    return audio

audio_file_path = "/content/audio1.wav"
audio_data = load_audio(audio_file_path)

if len(audio_data) > 0:
    sample_rate = 16000
    n_fft = 2048
    hop_length = 512
    n_mels = 128
    f_min = 0
    f_max = sample_rate

    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = mel_spectrogram.T

    output_file_path = "mel_spectrogram1.txt"
    with open(output_file_path, 'w') as output_file:
        output_file.write(str(mel_spectrogram.shape))

    print("Mel spectrogram saved to", output_file_path)

else:
    print("Failed to load audio file:", audio_file_path)


Mel spectrogram saved to mel_spectrogram1.txt


In [10]:
%%file testf.py
import pytest

def test_compare_shapes():
    file1 = 'mel_spectrogram.txt'
    file2 = 'mel_spectrogram1.txt'

    shape1 = open(file1, 'r').readline().strip()
    shape2 = open(file2, 'r').readline().strip()

    assert shape1 == shape2


Writing testf.py


In [11]:
!python -m pytest testf.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: anyio-3.7.1
collected 1 item                                                                                   

testf.py .                                                                                   [100%]

======================================== 1 passed in 0.01s =========================================
